# Limpieza de Datos de Información Planta Solar

In [1]:
#Imports
import pandas as pd
from IPython.display import Markdown

In [163]:
#Leemos la pestaña de Inyecciones
generacionRaw = pd.read_excel('../Datasets/Información Planta Solar.xlsx', 'Inyecciones')

In [30]:
#Renombramos las columnas para mejor lectura
generacionRaw.columns = [
    'Fecha', 'Verano', 'Bit Horario', 'Periodo', 'Generación kW', 'Radiacion Solar'
]
generacionRaw.head(4)

,Fecha,Verano,Bit Horario,Periodo,Generación kW,Radiacion Solar
0,2020-03-01 00:05:00,1,B,Del último domingo de octubre al sábado anteri...,0.0,NaN
1,2020-03-01 00:10:00,1,B,Del último domingo de octubre al sábado anteri...,0.0,NaN
2,2020-03-01 00:15:00,1,B,Del último domingo de octubre al sábado anteri...,0.0,NaN
3,2020-03-01 00:20:00,1,B,Del último domingo de octubre al sábado anteri...,0.0,NaN


In [133]:

#Aplicamos lo mismo, pero en la pestaña de Condiciones Ambientales
condicionesAmbientalesRaw = pd.read_excel('../Datasets/Información Planta Solar.xlsx', 'Condiciones ambientales')

In [134]:
#Renombramos las columnas para facilidad de lectura
condicionesAmbientalesRaw.columns = [
    'Ciudad', 'Dia', 'Hora', 'NumHora', 'Temperatura', 'PresionAtmosferica', 'ProbabilidadLluvia',
    'HumedadRelativa', 'VelocidadViento', 'DireccionViento', 'DireccionViento2', 'CoberturaNubes', 'IndiceUV', 'CondicionCielo',
    'DescDireccionViento', 'VelocidadRafaga', 'Precipitacion', 'Real'
]

In [135]:
condicionesAmbientalesRaw.head(4)

,Ciudad,Dia,Hora,NumHora,Temperatura,PresionAtmosferica,ProbabilidadLluvia,HumedadRelativa,VelocidadViento,DireccionViento,DireccionViento2,CoberturaNubes,IndiceUV,CondicionCielo,DescDireccionViento,VelocidadRafaga,Precipitacion,Real
0,Villa de Arista,2020-05-19,2020-05-19 08:00:00,8,19.36,NaN,0.0,73.20,2.90,66.80,NaN,NaN,NaN,Despejado,Noreste,4.00,0.0,0.0
1,Villa de Arista,2020-05-19,2020-05-19 09:00:00,9,22.11,NaN,0.0,65.23,2.64,175.28,NaN,NaN,NaN,Medio nublado,Sur,4.08,0.0,0.0
2,Villa de Arista,2020-05-19,2020-05-19 10:00:00,10,25.13,NaN,0.0,55.31,1.98,152.60,NaN,NaN,NaN,Medio nublado,Sureste,4.84,0.0,0.0
3,Villa de Arista,2020-05-19,2020-05-19 11:00:00,11,27.45,NaN,0.0,44.47,1.95,255.26,NaN,NaN,NaN,Medio nublado,Oeste,6.96,0.0,0.0


In [136]:
#Vamos a saber cuántos nulos hay en las columnas
generacionRaw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368352 entries, 0 to 368351
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   Fecha            368352 non-null  datetime64[ns]
 1   Verano           368352 non-null  int64         
 2   Bit Horario      368352 non-null  object        
 3   Periodo          368352 non-null  object        
 4   Generación kW    368352 non-null  float64       
 5   Radiacion Solar  34270 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 16.9+ MB


Como hay muchos nulos en la columna de Radiación Solar, la borramos

In [36]:
generacionMod = generacionRaw.drop(columns=['Radiacion Solar'])
generacionMod.head()

,Fecha,Verano,Bit Horario,Periodo,Generación kW
0,2020-03-01 00:05:00,1,B,Del último domingo de octubre al sábado anteri...,0.0
1,2020-03-01 00:10:00,1,B,Del último domingo de octubre al sábado anteri...,0.0
2,2020-03-01 00:15:00,1,B,Del último domingo de octubre al sábado anteri...,0.0
3,2020-03-01 00:20:00,1,B,Del último domingo de octubre al sábado anteri...,0.0
4,2020-03-01 00:25:00,1,B,Del último domingo de octubre al sábado anteri...,0.0


Ahora, con respecto a las condiciones ambientales, a partir de Julio 2022, se empezaron a descargar las condiciones reales, antes de eso, solo tenemos la historia de lo pronosticado. Entonces, tenemos que quedarnos principalmente con lo real y eliminar todo lo pronosticado que sí tenga su registro real.

In [137]:
#Todos los nulos en Real, se cambian a 0
condicionesAmbientalesMod = condicionesAmbientalesRaw.copy()
condicionesAmbientalesMod['Real'] = condicionesAmbientalesMod['Real'].fillna(0)

In [138]:
#Buscamos qué registros tienen condiciones reales
condicionesReales = condicionesAmbientalesMod.loc[condicionesAmbientalesRaw['Real'] == 1]

In [139]:
#Buscamos las filas pronosticadas cuyo registro real ya exista
filasRepetidas = condicionesAmbientalesMod.loc[condicionesAmbientalesMod['Hora'].isin(condicionesReales['Hora'])].loc[condicionesAmbientalesMod['Real'] == 0]

In [140]:
#Eliminamos las condiciones ambientales repetidas
condicionesAmbientalesMod = condicionesAmbientalesMod.drop(index=filasRepetidas.index.values.tolist())

In [141]:
#Vamos a ver los valores nulos
condicionesAmbientalesMod.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28192 entries, 0 to 35125
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Ciudad               28192 non-null  object        
 1   Dia                  28192 non-null  datetime64[ns]
 2   Hora                 28192 non-null  datetime64[ns]
 3   NumHora              28192 non-null  int64         
 4   Temperatura          28192 non-null  float64       
 5   PresionAtmosferica   501 non-null    float64       
 6   ProbabilidadLluvia   28191 non-null  float64       
 7   HumedadRelativa      28189 non-null  float64       
 8   VelocidadViento      28192 non-null  float64       
 9   DireccionViento      27690 non-null  float64       
 10  DireccionViento2     0 non-null      float64       
 11  CoberturaNubes       11042 non-null  float64       
 12  IndiceUV             5642 non-null   float64       
 13  CondicionCielo       28192 non-null 

Vamos a eliminar las columnas que tienen muchos nulos, ya que no nos sirven los datos.

In [142]:
condicionesAmbientalesMod = condicionesAmbientalesMod.drop(columns=['PresionAtmosferica', 'DireccionViento2', 'CoberturaNubes', 'IndiceUV'])

Como hay registros que tienen nulos, en el caso de los flotantes, vamos a interpolar.

In [143]:
condicionesAmbientalesMod['ProbabilidadLluvia'] = condicionesAmbientalesMod['ProbabilidadLluvia'].interpolate(method='linear')
condicionesAmbientalesMod['HumedadRelativa'] = condicionesAmbientalesMod['HumedadRelativa'].interpolate(method='linear')
condicionesAmbientalesMod['VelocidadViento'] = condicionesAmbientalesMod['VelocidadViento'].interpolate(method='linear')
condicionesAmbientalesMod['DireccionViento'] = condicionesAmbientalesMod['DireccionViento'].interpolate(method='linear')
condicionesAmbientalesMod['VelocidadRafaga'] = condicionesAmbientalesMod['VelocidadRafaga'].interpolate(method='linear')
condicionesAmbientalesMod['Precipitacion'] = condicionesAmbientalesMod['Precipitacion'].interpolate(method='linear')

In [144]:
#Comprobamos que ya no hay valores nulos
condicionesAmbientalesMod.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28192 entries, 0 to 35125
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Ciudad               28192 non-null  object        
 1   Dia                  28192 non-null  datetime64[ns]
 2   Hora                 28192 non-null  datetime64[ns]
 3   NumHora              28192 non-null  int64         
 4   Temperatura          28192 non-null  float64       
 5   ProbabilidadLluvia   28192 non-null  float64       
 6   HumedadRelativa      28192 non-null  float64       
 7   VelocidadViento      28192 non-null  float64       
 8   DireccionViento      28192 non-null  float64       
 9   CondicionCielo       28192 non-null  object        
 10  DescDireccionViento  28192 non-null  object        
 11  VelocidadRafaga      28192 non-null  float64       
 12  Precipitacion        28192 non-null  float64       
 13  Real                 28192 non-null 

Por último, vamos a unificar la generación y sus condiciones en un solo dataframe

In [145]:
#Obtenemos la hora a la que pertenece la fecha cincominutal de la generación
#Le sumamos 1 hora porque en condiciones comienza con la hora 1
#También el día, para que funcione el join
generacionMod['Hora'] = generacionMod['Fecha'].dt.hour + 1
generacionMod['Dia'] = pd.to_datetime(generacionMod['Fecha']).dt.date
generacionMod['Dia'] = pd.to_datetime(generacionMod['Dia'])

In [148]:
#Realizamos el join entre las dos tablas para unificarlas
GeneracionPanelSolar = generacionMod.merge(condicionesAmbientalesMod, left_on=['Hora', 'Dia'], right_on=['NumHora', 'Dia'], how='left')

In [149]:
#Revisamos que no haya nulos y que los tipos sean correctos
GeneracionPanelSolar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368772 entries, 0 to 368771
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Fecha                368772 non-null  datetime64[ns]
 1   Verano               368772 non-null  int64         
 2   Bit Horario          368772 non-null  object        
 3   Periodo              368772 non-null  object        
 4   Generación kW        368772 non-null  float64       
 5   Hora_x               368772 non-null  int32         
 6   Dia                  368772 non-null  datetime64[ns]
 7   Ciudad               331369 non-null  object        
 8   Hora_y               331369 non-null  datetime64[ns]
 9   NumHora              331369 non-null  float64       
 10  Temperatura          331369 non-null  float64       
 11  ProbabilidadLluvia   331369 non-null  float64       
 12  HumedadRelativa      331369 non-null  float64       
 13  VelocidadVient

In [153]:
#Eliminamos las generaciones que no tienen informacion de condiciones
GeneracionPanelSolar = GeneracionPanelSolar.dropna()

In [156]:
#Eliminamos columnas innecesarias
GeneracionPanelSolar = GeneracionPanelSolar.drop(columns=['Hora_x', 'Hora_y'])

In [158]:
#Esta es la tabla final limpia
GeneracionPanelSolar.head()

,Fecha,Verano,Bit Horario,Periodo,Generación kW,Dia,Ciudad,NumHora,Temperatura,ProbabilidadLluvia,HumedadRelativa,VelocidadViento,DireccionViento,CondicionCielo,DescDireccionViento,VelocidadRafaga,Precipitacion,Real
22823,2020-05-19 07:00:00,0,I,Del primer domingo de abril al sábado anterior...,6755.974734,2020-05-19,Villa de Arista,8.0,19.36,0.0,73.2,2.9,66.8,Despejado,Noreste,4.0,0.0,0.0
22824,2020-05-19 07:05:00,0,I,Del primer domingo de abril al sábado anterior...,4638.326658,2020-05-19,Villa de Arista,8.0,19.36,0.0,73.2,2.9,66.8,Despejado,Noreste,4.0,0.0,0.0
22825,2020-05-19 07:10:00,0,I,Del primer domingo de abril al sábado anterior...,5513.380467,2020-05-19,Villa de Arista,8.0,19.36,0.0,73.2,2.9,66.8,Despejado,Noreste,4.0,0.0,0.0
22826,2020-05-19 07:15:00,0,I,Del primer domingo de abril al sábado anterior...,5823.761920,2020-05-19,Villa de Arista,8.0,19.36,0.0,73.2,2.9,66.8,Despejado,Noreste,4.0,0.0,0.0
22827,2020-05-19 07:20:00,0,I,Del primer domingo de abril al sábado anterior...,4365.792235,2020-05-19,Villa de Arista,8.0,19.36,0.0,73.2,2.9,66.8,Despejado,Noreste,4.0,0.0,0.0


In [160]:
#Guardamos en un excel
GeneracionPanelSolar.to_excel('../Datasets/Informacion Planta Solar Limpio.xlsx', index=False)